<p style="font-size:200%; font-weight:bold"> Daily Transaction </p>

- Author: Yichen Luo
- Date: 20221005

In [1]:
import pandas as pd
import threading
import requests
import time
import os
import datetime
import re
import string
import random
import numpy as np
from web3 import Web3
from hexbytes import HexBytes
import seaborn as sns ; sns.set_theme(color_codes=True)

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

from IPython.core.interactiveshell import InteractiveShell
pd.set_option('max_colwidth', 100)
pd.set_option('display.max_columns', 100)

RD = '/home/team_defi/02_rdata/'
WD = '/home/team_defi/03_wdata/'
RES = '/home/team_defi/04_result/'
InteractiveShell.ast_node_interactivity = 'all'

# Example

In [3]:
df = pd.read_csv(f'{RES}/transaction/root.csv')
df['dex'].value_counts()
df.loc[df['dex'] == 'pancake_binance'].sample(1)

pancake_binance        12683
uniswapv2_ethereum      1865
apeswap_binance          367
pangolin_avalanche       285
traderjoe_avalanche      110
uniswapv3_ethereum        96
sushiswap_ethereum        20
babyswap_binance          18
biswap_binance            16
spookyswap_fantom          4
sushiswap_polygon          4
spiritswap_fantom          2
Name: dex, dtype: int64

,Unnamed: 0,amountUSD,id,logIndex,origin,recipient,sender,sqrtPriceX96,tick,timestamp,token0_name,token1_name,token0_id,token1_id,transaction_id,amount0Out,amount0In,amount1Out,amount1In,home_token,dex,chain,insider_root,contractAddress,from,to,lp,buy,ret_1,ret_2,ret_3,ret_-1,ret_-2,ret_-3
9818,7837,3110.03451,0xaa60c7008ae83b673a4b2d0f6ec88903ea462737d6ccb13a2f9af8688c42bf8b-0,1719,NaN,NaN,0x10ed43c718714eb63d5aa57b78b54704e256024e,NaN,NaN,2021-11-02 12:36:37,DogeDash,Wrapped BNB,0x7ae5709c585ccfb3e61ff312ec632c21a5f03f70,0xbb4cdb9cbd36b01bd1cbaebf2de08d9173bc095c,0xaa60c7008ae83b673a4b2d0f6ec88903ea462737d6ccb13a2f9af8688c42bf8b,0.0,4.036188e+06,5.692592,0.0,True,pancake_binance,binance,0x284aceea3a83322c4bc221954a6e2f51ea3e0acd,0x7ae5709c585ccfb3e61ff312ec632c21a5f03f70,0x284aceea3a83322c4bc221954a6e2f51ea3e0acd,0x10ed43c718714eb63d5aa57b78b54704e256024e,False,-3110.03451,-0.071784,-0.115596,-0.029178,0.41491,0.349565,0.337519


In [14]:
ID = '0x8e87a6ebfa4832995dcdec8197898f5dfb0a2af45be2439f7b4f8941e8b0e61f-0'
contractAddress = '0xd0f05d3d4e4d1243ac826d8c6171180c58eaa9bc'
timestamp = '2021-09-27 07:00:36'
timestamp = pd.to_datetime(timestamp).strftime('%Y-%m-%d')
start_time = pd.to_datetime(timestamp)
end_time = pd.to_datetime(timestamp) + datetime.timedelta(days=1)
start_unix = int(start_time.timestamp())
end_unix = int(end_time.timestamp())

start_unix
end_unix
int(pd.to_datetime('2021-09-27 07:00:36').timestamp())

1632700800

1632787200

1632726036

In [11]:
dfs = []

for token in ['token0', 'token1']:
    length = 1000
    unix = start_unix
    while length == 1000:
        url = 'https://api.thegraph.com/subgraphs/name/ianlapham/uniswapv2'

        query = string.Template("""query MyQuery {
  swaps(first: 1000, orderBy: timestamp, orderDirection: asc, where: {pair_: {$token: "$contractAddress"}, timestamp_gt: "$timestamp"}) {
    amount0In
    amount0Out
    amount1In
    amount1Out
    amountUSD
    from
    id
    logIndex
    to
    timestamp
    sender
    pair {
      token0 {
        id
        name
      }
      token1 {
        id
        name
      }
    }
    transaction {
      id
    }
  }
}""")
        query.substitute({'timestamp': unix, 'token':token, 'contractAddress': contractAddress})
        post = {"query":query.substitute({'timestamp': unix, 'token':token, 'contractAddress': contractAddress})}

        r = requests.post(url, json=post)
        data = r.json()
        df = pd.DataFrame.from_dict(data['data']['swaps'])
        if df.shape[0] == 0: 
            length = 0
            continue

        # df['token0_name'] = df['token0'].apply(lambda x: x['name'])
        # df['token1_name'] = df['token1'].apply(lambda x: x['name'])
        # df['token0_id'] = df['token0'].apply(lambda x: x['id'])
        # df['token1_id'] = df['token1'].apply(lambda x: x['id'])
        # df['transaction_id'] = df['transaction'].apply(lambda x: x['id'])
        # df.drop(columns=['token0', 'token1', 'transaction'], inplace=True)
        # df = uniswapv3_correction(df)

        df['token0_name'] = df['pair'].apply(lambda x: x['token0']['name'])
        df['token1_name'] = df['pair'].apply(lambda x: x['token1']['name'])
        df['token0_id'] = df['pair'].apply(lambda x: x['token0']['id'])
        df['token1_id'] = df['pair'].apply(lambda x: x['token1']['id'])
        df['transaction_id'] = df['transaction'].apply(lambda x: x['id'])
        df.drop(columns=['pair', 'transaction'], inplace=True)
        
        df
        df = df.loc[df['timestamp'].apply(lambda x: int(x)) <= end_unix]
        if df.shape[0] == 0: 
            length = 0
            continue
        unix = df.tail(1)['timestamp'].values[0]
        length = len(df)
        length
        
        # df['timestamp'] = pd.to_datetime(df['timestamp'],unit='s', origin='unix')
        unix
        dfs.append(df)

if len(dfs) != 0: 
    dfs = pd.concat(dfs)
    dfs.drop_duplicates('id', inplace=True)
    dfs.sort_values('timestamp', ascending=True, inplace=True)

'query MyQuery {\n  swaps(first: 1000, orderBy: timestamp, orderDirection: asc, where: {pair_: {token0: "0xd0f05d3d4e4d1243ac826d8c6171180c58eaa9bc"}, timestamp_gt: "1610150400"}) {\n    amount0In\n    amount0Out\n    amount1In\n    amount1Out\n    amountUSD\n    from\n    id\n    logIndex\n    to\n    timestamp\n    sender\n    pair {\n      token0 {\n        id\n        name\n      }\n      token1 {\n        id\n        name\n      }\n    }\n    transaction {\n      id\n    }\n  }\n}'

JSONDecodeError: [Errno Expecting value] <!DOCTYPE html>
<!--[if lt IE 7]> <html class="no-js ie6 oldie" lang="en-US"> <![endif]-->
<!--[if IE 7]>    <html class="no-js ie7 oldie" lang="en-US"> <![endif]-->
<!--[if IE 8]>    <html class="no-js ie8 oldie" lang="en-US"> <![endif]-->
<!--[if gt IE 8]><!--> <html class="no-js" lang="en-US"> <!--<![endif]-->
<head>
<meta http-equiv="refresh" content="0">

<title>api.thegraph.com | 524: A timeout occurred</title>
<meta charset="UTF-8" />
<meta http-equiv="Content-Type" content="text/html; charset=UTF-8" />
<meta http-equiv="X-UA-Compatible" content="IE=Edge" />
<meta name="robots" content="noindex, nofollow" />
<meta name="viewport" content="width=device-width,initial-scale=1" />
<link rel="stylesheet" id="cf_styles-css" href="/cdn-cgi/styles/main.css" />


</head>
<body>
<div id="cf-wrapper">

    

    <div id="cf-error-details" class="p-0">
        <header class="mx-auto pt-10 lg:pt-6 lg:px-8 w-240 lg:w-full mb-8">
            <h1 class="

In [13]:
dfs

,amount0In,amount0Out,amount1In,amount1Out,amountUSD,from,id,logIndex,to,timestamp,sender,token0_name,token1_name,token0_id,token1_id,transaction_id
0,0.0114154,0,0,12.885481931290387908,524.646071944991561658100184159365,0x2cbc862857490c73d46f9f142dfd4b14df027d16,0x13ce2db963ef00c2dc4a4ebd36fe0dc4ab605340a0dee5c7cedc0491961d58b6-0,12,0x19f5421d99233a4c120559954039e696e2dd897a,1631232673,0x19f5421d99233a4c120559954039e696e2dd897a,Wrapped BTC,Wrapped AVAX,0x50b7545627a5162f82a992c33b87adc75187b218,0xb31f66aa3c1e785363f0875a1b74e27b85fd66c7,0x13ce2db963ef00c2dc4a4ebd36fe0dc4ab605340a0dee5c7cedc0491961d58b6
1,0.01785,0,0,20.117348849990389737,830.43355364270442611795551122799,0xb589ede79cfe3b9a0e7a8c8f5432274443b04e4a,0x8ab32d6cbb15572fb5017d61ae3d766e2df82d004e08769ac2e60455a57423e5-0,3,0x3f2aea38c8d47d14111e2760c9ab1bb78b4a42c5,1631233358,0x3f2aea38c8d47d14111e2760c9ab1bb78b4a42c5,Wrapped BTC,Wrapped AVAX,0x50b7545627a5162f82a992c33b87adc75187b218,0xb31f66aa3c1e785363f0875a1b74e27b85fd66c7,0x8ab32d6cbb15572fb5017d61ae3d766e2df82d004e08769ac2e60455a57423e5
2,0.008923,0,0,10.042084994902369819,414.6308766218328659376074009489768,0x98736135beadcc859ae644dd38fc555423228d94,0x76f4d496f2202bc01ee4f0142010ed376b9ac83a969f17ec2c6afd3db8c72115-0,4,0x98736135beadcc859ae644dd38fc555423228d94,1631233360,0xe54ca86531e17ef3616d22ca28b0d458b6c89106,Wrapped BTC,Wrapped AVAX,0x50b7545627a5162f82a992c33b87adc75187b218,0xb31f66aa3c1e785363f0875a1b74e27b85fd66c7,0x76f4d496f2202bc01ee4f0142010ed376b9ac83a969f17ec2c6afd3db8c72115
3,0.00892499,0,0,10.034788821854257209,414.9282539282777632399086747160093,0x98736135beadcc859ae644dd38fc555423228d94,0xa1d04ff07a1fdce6f7523078a0fdcfdad581f1311a904bc7c69debff7d713807-0,4,0x98736135beadcc859ae644dd38fc555423228d94,1631233400,0xe54ca86531e17ef3616d22ca28b0d458b6c89106,Wrapped BTC,Wrapped AVAX,0x50b7545627a5162f82a992c33b87adc75187b218,0xb31f66aa3c1e785363f0875a1b74e27b85fd66c7,0xa1d04ff07a1fdce6f7523078a0fdcfdad581f1311a904bc7c69debff7d713807
4,0.02561401,0,0,28.746245092791266107,1191.674533823252407159208860502266,0xb589ede79cfe3b9a0e7a8c8f5432274443b04e4a,0x53e699972a81326c85038497d5c9e8d999cc353292d42b5e1e753a269456b3d6-0,7,0x3f2aea38c8d47d14111e2760c9ab1bb78b4a42c5,1631233541,0x3f2aea38c8d47d14111e2760c9ab1bb78b4a42c5,Wrapped BTC,Wrapped AVAX,0x50b7545627a5162f82a992c33b87adc75187b218,0xb31f66aa3c1e785363f0875a1b74e27b85fd66c7,0x53e699972a81326c85038497d5c9e8d999cc353292d42b5e1e753a269456b3d6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
927,0.03463446,0,0,31.74,1547.636802982739520372214856106436,0x3f9a88cdf89d2570b6e540c4f6c044ac08666929,0xb1e20091146b5d141514babcbf73dcfedd991e373ba00b36b50a57141ceadc05-0,4,0xe9627d56b243a32b8ad2415e49a297983f9d45c5,1631318030,0xe54ca86531e17ef3616d22ca28b0d458b6c89106,Wrapped BTC,Wrapped AVAX,0x50b7545627a5162f82a992c33b87adc75187b218,0xb31f66aa3c1e785363f0875a1b74e27b85fd66c7,0xb1e20091146b5d141514babcbf73dcfedd991e373ba00b36b50a57141ceadc05
928,0,0.00957739,8.818166287335248416,0,429.1855898623518858248331447504222,0xfcff28c4a7338fc213923801c6b1d14a80143714,0x0dde29ca20992ddf2650c089bac70765dbc1e7a12456cf22c9b6d54adb786805-0,3,0xb8d5e8a9247db183847c7d79af9c67f6aef759f7,1631318031,0x1ac22c5cabf963141c8e9dc4d1f17b9ab488afa1,Wrapped BTC,Wrapped AVAX,0x50b7545627a5162f82a992c33b87adc75187b218,0xb31f66aa3c1e785363f0875a1b74e27b85fd66c7,0x0dde29ca20992ddf2650c089bac70765dbc1e7a12456cf22c9b6d54adb786805
930,0.00548849,0,0,128.400726657281972235,247.4400853215339806661281680942861,0xfcff28c4a7338fc213923801c6b1d14a80143714,0x0da01e3b768aa46ba5d74a725df96a7a6092fb1962e69ee7a9e50f25eafaf3e5-1,6,0xd7538cabbf8605bde1f4901b47b8d42c61de0367,1631318190,0x1ac22c5cabf963141c8e9dc4d1f17b9ab488afa1,Wrapped BTC,Pangolin,0x50b7545627a5162f82a992c33b87adc75187b218,0x60781c2586d68229fde47564546784ab3faca982,0x0da01e3b768aa46ba5d74a725df96a7a6092fb1962e69ee7a9e50f25eafaf3e5
929,0,0.00548849,5.057447815889471761,0,246.4315616612048487564562382363616,0xfcff28c4a7338fc

# Web Crawler

## List

In [13]:
# df = []
# panels = ['root', 'layer_one', 'whale_new']
# dfs = pd.read_csv(f'{WD}token_ret_new/lst/panel_insider.csv')

# for panel in panels:

#     df1 = dfs.loc[(dfs['panel'] == panel)&(dfs['insider_trading'] == True)]
#     df2 = pd.read_csv(f'{RES}/transaction/{panel}.csv')
#     if panel != 'root': df2.rename(columns={'insider_contractAddress':'contractAddress'}, inplace=True)
#     df2['timestamp'] = pd.to_datetime(df2['timestamp'])
#     df2['date'] = df2['timestamp'].dt.strftime('%Y-%m-%d')

#     for idx, row in df1.iterrows():
#         contractAddress = row['contractAddress']
#         date = row['date']
#         chain = row['chain']

#         csv = df2.loc[(df2['contractAddress'] == contractAddress)&(df2['date'] == date)&(df2['chain'] == chain)].copy()
#         csv['panels'] = panel
#         df.append(csv)

# df = pd.concat(df)
# df.to_csv(f'{WD}/daily_transaction/list/panel.csv', index=False)

/tmp/ipykernel_3029929/1786260112.py:8: DtypeWarning: Columns (3,4,6) have mixed types. Specify dtype option on import or set low_memory=False.
  df2 = pd.read_csv(f'{RES}/transaction/{panel}.csv')


In [10]:
# df = pd.read_csv(f'{WD}/daily_transaction/list/panel.csv')
# df.drop(columns=['Unnamed: 0'], inplace=True)
# df['contractaddress'] = df['contractAddress'].apply(lambda x: x.lower())
# df['status'] = -1
# for dex in df['dex'].unique():
#     df.loc[df['dex'] == dex].to_csv(f'{WD}/daily_transaction/list/{dex}.csv')

/tmp/ipykernel_2507097/2820005383.py:1: DtypeWarning: Columns (4,5,7,22,35,36) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f'{WD}/daily_transaction/list/panel.csv')


## Data

In [10]:
class web_thread(threading.Thread):
    
    def __init__ (self, dex):
        threading.Thread.__init__(self)
        self.dex = dex
        
    def run(self):
        RD = '/home/team_defi/02_rdata/'
        WD = '/home/team_defi/03_wdata/'
        RES = '/home/team_defi/04_result/'
        
        lst = pd.read_csv(f'{WD}daily_transaction/list/{self.dex}.csv')
        
        done = lst.loc[lst['status'] != -1].drop_duplicates(['contractaddress', 'date']).shape[0]
        total = lst.drop_duplicates(['contractaddress', 'date']).shape[0]
        
        for idx, row in lst.loc[lst['status'] == -1].drop_duplicates(['contractaddress', 'date']).iterrows():
            contractaddress = row['contractaddress']
            timestamp = row['timestamp']
            ID = row['id']
            date = row['date']
            
            try:
                done += 1
                
                dfs = insider_transaction(contractAddress = contractaddress, timestamp = timestamp, dex=self.dex)
                
                if len(dfs) == 0:
                    lst.loc[(lst['contractaddress'] == contractaddress)&(lst['date'] == date), 'status'] = 0
                    print(f'{contractaddress} at {timestamp} for {ID}, {self.dex} no swap, {done}/{total}')
                else:
                    dfs.to_csv(f'{WD}daily_transaction/csv/{contractaddress}_{date}_{self.dex}.csv', index=False)
                    dfs_length = dfs.shape[0]
                    lst.loc[(lst['contractaddress'] == contractaddress)&(lst['date'] == date), 'status'] = dfs_length
                    print(f'{contractaddress} at {timestamp} for {ID}, {self.dex} find {dfs_length} swap, {done}/{total}')
                
                lst.to_csv(f'{WD}daily_transaction/list/{self.dex}.csv', index=False)
                
            except Exception as e:
                print(f'{contractaddress} at {timestamp} for {ID}, {self.dex}, Error: {e}')
                
        print(f'{self.dex} done')

In [11]:
thread1 = web_thread('uniswapv3_ethereum')
thread2 = web_thread('uniswapv2_ethereum')
thread3 = web_thread('sushiswap_ethereum')

thread4 = web_thread('pancake_binance')
thread5 = web_thread('biswap_binance')
thread6 = web_thread('apeswap_binance')
thread7 = web_thread('babyswap_binance')
thread8 = web_thread('jetswap_binance')

thread9 = web_thread('traderjoe_avalanche')
thread10 = web_thread('pangolin_avalanche')

thread11 = web_thread('sushiswap_polygon')
thread12 = web_thread('apeswap_polygon')
thread13 = web_thread('jetswap_polygon')

thread14 = web_thread('spookyswap_fantom')
thread15 = web_thread('spiritswap_fantom')

In [12]:
Threads = [
           thread1, 
           thread2, 
           thread3,
           # thread4,
           thread5,
           # thread6,
           thread7,
           # thread8,
           thread9,
           thread10,
           thread11,
           # thread12,
           # thread13,
           thread14,
           thread15,
          ]
for t in Threads:
    t.start()
for t in Threads:
    t.join()

sushiswap_ethereum done
spiritswap_fantom done
spookyswap_fantom done
sushiswap_polygon done
biswap_binance done
babyswap_binance done
uniswapv3_ethereum done
pangolin_avalanche done
traderjoe_avalanche done


/tmp/ipykernel_3029929/1356528528.py:12: DtypeWarning: Columns (22,35) have mixed types. Specify dtype option on import or set low_memory=False.
  lst = pd.read_csv(f'{WD}daily_transaction/list/{self.dex}.csv')


0xe7ae6d0c56cacaf007b7e4d312f9af686a9e9a04 at 2022-07-18 19:23:53 for 0x6dafd7e6c0481ed9816a353778cde058190997e91d261512e05a040d6db73ef4-0, uniswapv2_ethereum find 9 swap, 10391/10391
uniswapv2_ethereum done


# Function

In [5]:
def insider_transaction(contractAddress, timestamp, dex):
    
    timestamp = pd.to_datetime(timestamp).strftime('%Y-%m-%d')
    start_time = pd.to_datetime(timestamp)
    end_time = pd.to_datetime(timestamp) + datetime.timedelta(days=1)
    start_unix = int(start_time.timestamp()) - 1
    end_unix = int(end_time.timestamp())

    dfs = []
    for token in ['token0', 'token1']:
        length = 1000
        unix = start_unix
        while length == 1000:
            post = {"query":dex_info(dex)['post'].substitute({'timestamp': unix, 'token':token, 'contractAddress': contractAddress})}
            url = dex_info(dex)['url']

            r = requests.post(url, json=post)
            data = r.json()
            df = pd.DataFrame.from_dict(data['data']['swaps'])
            if df.shape[0] == 0:
                length = 0
                continue

            if dex_info(dex)['style'] == 0:
                df['token0_name'] = df['token0'].apply(lambda x: x['name'])
                df['token1_name'] = df['token1'].apply(lambda x: x['name'])
                df['token0_id'] = df['token0'].apply(lambda x: x['id'])
                df['token1_id'] = df['token1'].apply(lambda x: x['id'])
                df['transaction_id'] = df['transaction'].apply(lambda x: x['id'])
                df.drop(columns=['token0', 'token1', 'transaction'], inplace=True)
            else:
                if dex_info(dex)['style'] == 1:
                    df['token0_name'] = df['pair'].apply(lambda x: x['token0']['name'])
                    df['token1_name'] = df['pair'].apply(lambda x: x['token1']['name'])
                    df['token0_id'] = df['pair'].apply(lambda x: x['token0']['id'])
                    df['token1_id'] = df['pair'].apply(lambda x: x['token1']['id'])
                    df['transaction_id'] = df['transaction'].apply(lambda x: x['id'])
                    df.drop(columns=['pair', 'transaction'], inplace=True)

            df = df.loc[df['timestamp'].apply(lambda x: int(x)) <= end_unix]
            
            if df.shape[0] == 0: 
                length = 0
                continue
            
            unix = df.tail(1)['timestamp'].values[0]
            length = len(df)
            
            df['timestamp'] = pd.to_datetime(df['timestamp'],unit='s', origin='unix')

            if dex == 'uniswapv3_ethereum': df = uniswapv3_correction(df)
            dfs.append(df)

    if len(dfs) != 0: 
        dfs = pd.concat(dfs)
        dfs.drop_duplicates('id', inplace=True)

    return dfs

In [6]:
def uniswapv3_correction(df):
    
    df['amount0'] = df['amount0'].astype('float')
    df['amount1'] = df['amount1'].astype('float')
    for idx, row in df.iterrows():
        if row['amount0'] > 0:
            df.loc[df['id'] == row['id'], 'amount0Out'] = 0
            df.loc[df['id'] == row['id'], 'amount0In'] = abs(row['amount0'])
            df.loc[df['id'] == row['id'], 'amount1Out'] = abs(row['amount1'])
            df.loc[df['id'] == row['id'], 'amount1In'] = 0

        else:
            df.loc[df['id'] == row['id'], 'amount0Out'] = abs(row['amount0'])
            df.loc[df['id'] == row['id'], 'amount0In'] = 0
            df.loc[df['id'] == row['id'], 'amount1Out'] = 0
            df.loc[df['id'] == row['id'], 'amount1In'] = abs(row['amount1'])
    df.drop(columns=['amount1', 'amount0'], inplace=True)
    return df

In [7]:
def dex_info(dex):
    dex_infos = {
                'uniswapv3_ethereum':{'url': 'https://api.thegraph.com/subgraphs/name/uniswap/uniswap-v3',
                                       'role': 1,
                                       'style': 0,
                                       'post': 1
                                      },
                
                'uniswapv2_ethereum':{'url': 'https://api.thegraph.com/subgraphs/name/ianlapham/uniswapv2',
                                       'role': 0,
                                       'style': 1,
                                       'post': 0
                                      },
        
                'sushiswap_ethereum':{'url': 'https://api.thegraph.com/subgraphs/name/sushiswap/exchange',
                                       'role': 2,
                                       'style': 1,
                                       'post': 2
                                      },
        
                'pancake_binance':{'url': 'https://bsc.streamingfast.io/subgraphs/name/pancakeswap/exchange-v2',
                                       'role': 0,
                                       'style': 1,
                                       'post': 3
                                      },
        
                'biswap_binance':{'url': 'https://api.thegraph.com/subgraphs/name/biswapcom/exchange5',
                                       'role': 0,
                                       'style': 1,
                                       'post': 0
                                      },
        
                'apeswap_binance':{'url': 'https://graph2.apeswap.finance/subgraphs/name/ape-swap/apeswap-subgraph',
                                       'role': 0,
                                       'style': 1,
                                       'post': 0
                                      },
        
                'babyswap_binance':{'url': 'https://api.thegraph.com/subgraphs/name/loremboard/bsc-babyswap-subgraph',
                                       'role': 0,
                                       'style': 1,
                                       'post': 0
                                      },
        
                'jetswap_binance':{'url': 'https://api.thegraph.com/subgraphs/name/smartcookie0501/jetswap-subgraph-v2',
                                       'role': 0,
                                       'style': 1,
                                       'post': 0
                                      },
        
                'traderjoe_avalanche':{'url': 'https://api.thegraph.com/subgraphs/name/traderjoe-xyz/exchange',
                                       'role': 2,
                                       'style': 1,
                                       'post': 2
                                      }, 
        
                'pangolin_avalanche':{'url': 'https://api.thegraph.com/subgraphs/name/dasconnor/pangolin-dex',
                                       'role': 0,
                                       'style': 1,
                                       'post': 0
                                      },  
        
                'spookyswap_fantom':{'url': 'https://api.thegraph.com/subgraphs/name/int3grtor/spooky-exchange',
                                       'role': 2,
                                       'style': 1,
                                       'post': 2
                                      },  
        
                'spiritswap_fantom':{'url': 'https://api.thegraph.com/subgraphs/name/evele/spirit-swap-subgraph-dev-v2',
                                       'role': 2,
                                       'style': 1,
                                       'post': 2
                                      },  
        
                'sushiswap_polygon':{'url': 'https://api.thegraph.com/subgraphs/name/sushiswap/matic-exchange',
                                       'role': 2,
                                       'style': 1,
                                       'post': 2
                                      },
        
                'apeswap_polygon':{'url': 'https://api.thegraph.com/subgraphs/name/apeswapfinance/dex-polygon',
                                       'role': 0,
                                       'style': 1,
                                       'post': 0
                                      },
        
                'jetswap_polygon':{'url': 'https://api.thegraph.com/subgraphs/name/smartcookie0501/jetswap-subgraph-polygon-v2',
                                       'role': 0,
                                       'style': 1,
                                       'post': 0
                                      },
                }
    
    roles = [['from', 'to', 'sender'],
             ['origin', 'recipient', 'sender'],
             ['sender', 'to']]
    
    url = dex_infos[dex]['url']
    role = roles[dex_infos[dex]['role']]
    style = dex_infos[dex]['style']
    post = dex_infos[dex]['post']
    
    info = {'url':url,
            'post':post_string(post),
            'role':role,
            'style':style
           }

    return info

In [8]:
def post_string(post):
    if post == 0:
        query = string.Template("""query MyQuery {
  swaps(first: 1000, orderBy: timestamp, orderDirection: asc, where: {pair_: {$token: "$contractAddress"}, timestamp_gt: "$timestamp"}) {
    amount0In
    amount0Out
    amount1In
    amount1Out
    amountUSD
    from
    id
    logIndex
    to
    timestamp
    sender
    pair {
      token0 {
        id
        name
      }
      token1 {
        id
        name
      }
    }
    transaction {
      id
    }
  }
}""")
        
    if post == 1:
        query = string.Template("""query MyQuery {
  swaps(first: 1000, orderDirection: asc, orderBy: timestamp, where: {timestamp_gt: "$timestamp", $token: "$contractAddress"}) {
    amount0
    amount1
    amountUSD
    id
    logIndex
    origin
    recipient
    sender
    sqrtPriceX96
    tick
    timestamp
    token0 {
      id
      name
    }
    token1 {
      id
      name
    }
    transaction {
      id
    }
  }
}""")
        
    if post == 2:
            query = string.Template("""query MyQuery {
  swaps(first: 1000, orderBy: timestamp, orderDirection: asc, where: {pair_: {$token: "$contractAddress"}, timestamp_gt: "$timestamp"}) {
    amount0In
    amount0Out
    amount1In
    amount1Out
    amountUSD
    id
    logIndex
    sender
    timestamp
    to
    pair {
      token0 {
        id
        name
      }
      token1 {
        id
        name
      }
    }
    transaction {
      id
    }
  }
}""")
    if post == 3:
        query = string.Template("""query MyQuery {
  swaps(first: 1000, orderBy: timestamp, orderDirection: asc, where: {timestamp_gt: "$timestamp", $token: "$contractAddress"}) {
    amount0In
    amount0Out
    amount1In
    amount1Out
    amountUSD
    from
    id
    logIndex
    to
    timestamp
    sender
    pair {
      token0 {
        id
        name
      }
      token1 {
        id
        name
      }
    }
    transaction {
      id
    }
  }
}""")
    return query

In [9]:
def chain_info(chain):
    dexes = {'ethereum':['uniswapv3_ethereum', 'uniswapv2_ethereum', 'sushiswap_ethereum'],
             'binance':['pancake_binance', 'biswap_binance', 'apeswap_binance', 'babyswap_binance', 'jetswap_binance'],
             'avalanche':['traderjoe_avalanche', 'pangolin_avalanche'],
             'fantom':['spookyswap_fantom', 'spiritswap_fantom'],
             'polygon':['sushiswap_polygon', 'apeswap_polygon', 'jetswap_polygon']
            }
    
    dex = dexes[chain]
    return dex